In [17]:
import psycopg2
import pandas as pd


conn = psycopg2.connect(
    host="0.0.0.0",
    database="mydb",
    user="sammam",
    password="mysecretpassword"
)

In [18]:
sql_query = "SELECT * FROM car_details INNER JOIN emission_standard ON car_details.stnd = emission_standard.stnd"

df = pd.read_sql(sql_query, conn)

conn.close()

/var/folders/6g/4zq1d985075_7_l6gxj2n81m0000gn/T/ipykernel_5593/808618032.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


In [27]:
print(df)

      model  disp  cyl  trans  drive  fuel  veh_class  air_pollution_score  \
0         3   3.0  6.0      1      1     2          4                  3.0   
1         0   2.4  4.0      2      0     2          5                  3.0   
2         0   2.4  4.0      2      0     2          5                  3.0   
3         3   3.5  6.0     29      0     2          4                  3.0   
4         3   3.5  6.0     29      0     2          4                  3.0   
...     ...   ...  ...    ...    ...   ...        ...                  ...   
4135    714   4.0  8.0     28      1     2          7                  3.0   
4136    748   6.2  8.0     10      1     2          3                  1.0   
4137    748   6.2  8.0     10      1     2          3                  1.0   
4138    792   1.6  3.0     17      1     2          5                  5.0   
4139    833   1.5  4.0     28      0     2          4                  7.0   

      city_mpg  city_mpg_alternate  hwy_mpg  hwy_mpg_alternate 

In [28]:
print(df.describe())

             model         disp          cyl        trans        drive  \
count  4140.000000  4140.000000  4140.000000  4140.000000  4140.000000   
mean    408.922464     2.948671     5.402899    17.921014     0.471739   
std     223.392097     1.301201     1.932040     9.276655     0.499261   
min       0.000000     0.600000     2.000000     0.000000     0.000000   
25%     241.000000     2.000000     4.000000    11.000000     0.000000   
50%     373.000000     2.500000     4.000000    19.000000     0.000000   
75%     587.250000     3.500000     6.000000    28.000000     1.000000   
max     860.000000     8.000000    16.000000    29.000000     1.000000   

              fuel    veh_class  air_pollution_score     city_mpg  \
count  4140.000000  4140.000000          4140.000000  4140.000000   
mean      2.028502     3.880676             4.861353    22.139855   
std       0.396480     2.264523             1.758452     7.877843   
min       0.000000     0.000000             1.000000     

Preprocessing

In [20]:
df = df.drop(columns=['underhood_id', 'stnd_description', 'stnd'])

In [21]:
from sklearn.preprocessing import LabelEncoder
print(df.columns)

def label_encode_columns(df, column_names):

    df_encoded = df.copy()
    encoder = LabelEncoder()

    for col in column_names:
        try:
            if col in df_encoded.columns:
                df_encoded[col] = encoder.fit_transform(df_encoded[col])
        except:
            print(col)
    return df_encoded

cate_cols = ['stnd', 'model', 'trans', 'drive', 'fuel', 'veh_class', 'smart_way', 'cert_region']

df = label_encode_columns(df, cate_cols)


Index(['model', 'disp', 'cyl', 'trans', 'drive', 'fuel', 'veh_class',
       'air_pollution_score', 'city_mpg', 'city_mpg_alternate', 'hwy_mpg',
       'hwy_mpg_alternate', 'cmb_mpg', 'cmb_mpg_alternate',
       'greenhouse_gas_score', 'smart_way', 'comb_co2', 'comb_co2_alternate',
       'cert_region'],
      dtype='object')


In [83]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, cross_val_score


X = df.drop(columns=['comb_co2', 'comb_co2_alternate'])
y = df[['comb_co2', 'comb_co2_alternate']]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(n_estimators = 21, max_features = 'sqrt', max_depth = 30, random_state = 42)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

rmse = mse**.5
print("Root Mean Squared Error:", rmse)


Mean Squared Error: 15.993063010001888
Root Mean Squared Error: 3.9991327822419063
